<H1>Economic Lot Sizing</H1>

Consider a discrete-time, finite-horizon inventory control problem where you must satisfy a given demand. $d_t$ for each week $t$. This demand can be satisfied by production in any week $t' \le t$. For each week, there is a per unit production cost, a per unit inventory holding cost, and a setup cost. The setup cost is a fixed cost incurred if any production occurs during the week. 

Since the setup cost is paid at most once each week, regardless of whether you produce 1 unit or 1000, you have an incentive to potentially produce enough during a week to cover the demand for multiple weeks. But, producing in excess of the week's demand requires you to hold inventory. This tradeoff between setup and holding costs can be managed through optimization.

<H2>Initial Formulation</H2>
We'll consider two formulations of the problem. The first has fewer decision variables but has a drawback that we'll soon discover.

<H3>Sets and Indices</H3>
$t \in \{1,\ldots, T\}$: weeks

<H3>Data</H3>
$c_t$: per unit cost of production in week $t$

$h_t$: per unit holding cost for inventory remaining in week $t$

$f_t$: fixed cost of production in week $t$

$d_t$: demand for week $t$

<H3>Decision Variables</H3>

$x_t$: binary variable to indicate whether production occurs in week $t$

$y_t$: production quantity for week $t$

$s_t$: inventory carried in to period $t$

<H3>Formulation</H3>
\begin{eqnarray}
\min_{x, y, s} && \sum_{t = 1}^T (f_t x_t + c_t y_t + h_t s_t) \nonumber \\
\mbox{s.t.} && y_t \le (\sum_{t' = t}^{T} d_{t'}) x_t, \;\;t = 1,\ldots, T\nonumber \\
&& s_t = s_{t-1} + y_t - d_t,\;\;t = 1,\ldots,T \nonumber \\
&& s_0 = 0 \nonumber \\
&& x_t \in \{0, 1\},\;\;t = 1,\ldots,T \nonumber \\
&& y_t \ge 0,\;\; t = 1,\ldots,T \nonumber \\
&& s_t \ge 0,\;\; t = 1,\ldots,T \nonumber \\
\end{eqnarray}


In [ ]:
demands = [20, 50, 10, 50, 50, 10, 20, 40, 20, 30]

In [ ]:
production_costs = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [ ]:
fixed_costs = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

In [ ]:
holding_costs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
import gurobipy as grb
GRB = grb.GRB

def get_lot_sizes(demands, production_costs, fixed_costs, holding_costs):
    m = grb.Model()
    
    weeks = range(len(demands))
    production_quantity = m.addVars(weeks, name='production_quantity')
    holding_quantity = m.addVars(weeks, name='hold')
    to_produce = m.addVars(weeks, name='to_produce', vtype=GRB.BINARY)
    m.update()
    
    for week in weeks:
        #Add a constraint that ensures that production_quantity is only nonzero in weeks where to_produce is also nonzero
        m.addConstr(...)
        carried_inventory = holding_quantity[week-1] if week > 0 else 0
        #Add an inventory flow constraint that maintains an appropriate holding_quantity value for each week
        m.addConstr(...)
    
    total_production_cost = grb.quicksum(production_costs[week]*production_quantity[week]
                                         for week in weeks)
    total_fixed_cost = grb.quicksum(fixed_costs[week]*to_produce[week]
                                    for week in weeks)
    total_holding_cost = grb.quicksum(holding_costs[week]*holding_quantity[week]
                                      for week in weeks)
    m.setObjective(total_production_cost + total_fixed_cost + total_holding_cost)

    m.optimize()
    
    if m.Status == GRB.OPTIMAL:
        for week, var in enumerate(production_quantity):
            if var.X > 1e-4:
                print ("produce", var.X, "units in week", week)

    

In [ ]:
get_lot_sizes(demands, production_costs, fixed_costs, holding_costs)

It is therefore optimal to produce 80 units in week 0 to cover demand through week 2, to produce 130 units in week 3 to cover demand for weeks 3 through 6, and to produce 90 units in week 7 to cover demand for weeks 7 through 9.

Note that the LP relaxation produced a lower bound on the cost of 3375.304, but ultimately that lower bound could not be acheived as the true minimum cost was 3580. It is possible to formulate the problem in a way that the LP relaxation value is tight, and we can obtain the optimal integer solution immediately without resorting to branch and bound.

<H2>A Tighter Formulation</H2>

We'll consider a reformulation of the problem which has quadratically many variables, but that produces an LP relaxation that is tight. While the problem size will increase, this model will scale better since it produces an integer feasible solution immediately upon solving the LP relaxation.

The trick here is to note that production in week $t$ may be used to satisfy demand in any week $t' \ge t$. We can define the production variables to include both a production week $t$ and a demand week $t'$. We'll need to adjust the cost of these variables to include both the variable cost of production and the cost to hold inventory from week $t$ into week $t'$. This adjusted cost is simply $c_t + \sum_{i = t}^{t'-1} h_i$.

<H3>Data</H3>
* $\bar{c}_{tt'}$: cost to produce a unit in week $t$ and hold it as inventory until week $t'$
* $f_t$: fixed cost of production in week $t$

<H3>Decision Variables</H3>
* $x_t$: binary variable to indicate whether production occurs in week $t$
* $y_{tt'}$: number of units produced in week $t$ to satisfy demand in week $t' \ge t$

<H3>Formulation</H3>
\begin{eqnarray}
\min_{x, y} && \sum_{t = 1}^T \sum_{t'=t}^T \bar{c}_{tt'} y_{tt'} + \sum_{t = 1}^T f_t x_t \nonumber \\
\mbox{s.t.} && \sum_{t = 1}^{t'} y_{tt'} = d_{t'},\;\;t' = 1,\ldots, T \nonumber \\
&& y_{tt'} \le d_{t'}x_{t},\;\;t' = 1,\ldots, T,\;\;t=1,\ldots,t' \nonumber \\
&& x_t \in \{0, 1\},\;\;t = 1,\ldots, T \nonumber \\
&& y_{tt'} \ge 0,\;\;t = 1,\ldots, T,\;\;t' = t,\ldots,|T|. \nonumber
\end{eqnarray}

In [ ]:
def get_lot_sizes(demands, production_costs, fixed_costs, holding_costs):
    def get_cost(production_week, demand_week):
        # return the cost of producing a unit in production_week and 
        # holding it as inventory until demand_week
        return ...
    
    m = grb.Model()
    weeks = range(len(demands))
    costs = {(production_week, demand_week): get_cost(production_week, demand_week)  
            for production_week in weeks
            for demand_week in weeks[production_week:]}

    production_quantity = m.addVars(costs, obj=costs, name="total_cost")
    to_produce = m.addVars(weeks, obj=fixed_costs, name='to_produce', vtype=GRB.BINARY)
    m.update()

    for demand_week in weeks:
        # add a constraint to ensure that demand is satisfied in demand_week
        # hint: you can use production_quantity.sum to obtain an expression for the
        # quantity produced to satisfy demand in demand_week
        m.addConstr(...)
    
    for (production_week, demand_week), production_var in production_quantity.items():
        # add a constraint to enforce that production_var is 0 unless the 
        # corresponding to_produce variable is 1
        m.addConstr(...)

    m.optimize()

    if m.Status == GRB.OPTIMAL:
        # iterate over the sets of decision variables and print the recommendations
        
    m.write('lot_sizing.lp')
    m.write('lot_sizing.sol')